## Seed selection

In [ ]:
import gensim
gensim.__version__

In [ ]:
import nltk
nltk.__version__

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
global_glove_file = '../CI_Policy_Annotation/Flask/static/glove.6B/glove.6B.50d.txt'
glove2word2vec(glove_input_file=global_glove_file, word2vec_output_file="gensim_glove_vectors.txt")

from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)

In [ ]:
import pandas as pd
df = pd.read_csv('fews-district-all.csv')
countries = df['country'].unique()

In [ ]:
countries

In [ ]:
from collections import Counter

def bigram_sequence(text_lst):
    result = [a for ls in text_lst for a in zip(ls.split(" ")[:-1], ls.split(" ")[1:]) if 'food' in a or 'hunger' in a]
    return result

def trigram_sequence(text_lst):
    result = [a for ls in text_lst for a in zip(ls.split(" ")[:-2], ls.split(" ")[1:-1], ls.split(" ")[2:]) if 'food' in a or 'hunger' in a]
    return result

def get_words(country):
    words = Counter([])
    for t in range(25):
        # Post-processed Factiva avro files into csv
        df = pd.read_csv("./templates-v3/{}/tmp{}.csv".format(country, str(t).zfill(2)))
        for r in df.iterrows():
            unigrams = set(r[1]['body'].split())
            bigrams = set(bigram_sequence(r[1]['body'].split('.')))
            trigrams = set(trigram_sequence(r[1]['body'].split('.')))
            words.update(unigrams)
            words.update(bigrams)
            words.update(trigrams)
    return words

all_words = Counter([])
for country in countries:
    all_words.update(get_words(country))
select_df = pd.DataFrame()
select_df['seed'] = list(all_words)
select_df.to_csv('selected_words.csv')

In [ ]:
# Add unigrams, bigrams and trigrams with word 'food' and 'hunger'
seeds = pd.read_csv('selected_words.csv')
keys = ['hunger crises', 'famine', 'food insecurity']
seeds['distance'] = seeds['seed'].apply(lambda x: min([glove_model.wmdistance(x, k) for k in keys]))

# Including the 3 keys, retain just the top 100
seeds.sort_values(by=['distance'])[:103].to_csv('seeds_with_distance.csv')

In [ ]:
seeds['distance'].describe()